In [1]:
from sympy import *


In [2]:
# Variables
t, n = symbols('t n', integer=True)  # t: current iteration, n: desired iteration
x = Function('x')(t)  # Symbolic function for x
uf = Function('u')(t)
u = IndexedBase('u')
v = symbols('v')

In [3]:
def expr_u(t):
    return u[t]

In [4]:


# Functions
f = Function('f')(x)  # Symbolic function for f(x)
g = Function('g')(u)  # Symbolic function for g(u)
g = Function('h')(x,u)  # Symbolic function for g(u)


In [5]:
x0 = symbols('x0')
x_t0 = x.subs(t, 0).subs(x, x0)  # x_t0 represents x at time t=0


In [6]:
a = symbols("a")
b = symbols("b")

alpha = symbols(r"\alpha")
K = symbols('K')

In [13]:
# Given functions (replace with your actual functions)
f =  (b/ K)* x * (K-x) 
g =   (x* (u+b)**2)
h =  0

# Define initial condition
x0 = 1

x_t1 = f + g + h # x at time t+1
x_t1 = x_t1.simplify() 

# Define the desired iteration
desired_iteration = 3

# Apply the code
x_n = x_t0  # Initial condition for the desired iteration
for i in range(1,desired_iteration+1):
    x_n = x_t1.subs({x : x_n, u:uf.subs(t,i)}) + x_n * uf.subs(t,i-1)

# Display the result
print(f"The value of x at iteration {desired_iteration} is:")


The value of x at iteration 3 is:


In [14]:
print( uf.subs(t, desired_iteration-1) , '=', solve(x_n, uf.subs(t, desired_iteration-1))[0])

u(2) = (-K*b - sqrt(-K**2*b - K**2*u(1) + K*b**3*x(0) + 2*K*b**2*u(1)*x(0) + K*b**2*x(0) + K*b*u(0)*x(0) + K*b*u(1)**2*x(0) - b**2*x(0)**2))/K


In [15]:
solve(x_n, uf.subs(t,desired_iteration))[0]

-b - sqrt(-K**4*b - K**4*u(2) + K**3*b**5*x(0) + 2*K**3*b**4*u(1)*x(0) + 2*K**3*b**4*u(2)*x(0) + 2*K**3*b**4*x(0) + K**3*b**3*u(0)*x(0) + K**3*b**3*u(1)**2*x(0) + 4*K**3*b**3*u(1)*u(2)*x(0) + 3*K**3*b**3*u(1)*x(0) + K**3*b**3*u(2)**2*x(0) + 2*K**3*b**3*u(2)*x(0) + K**3*b**3*x(0) + 2*K**3*b**2*u(0)*u(2)*x(0) + K**3*b**2*u(0)*x(0) + 2*K**3*b**2*u(1)**2*u(2)*x(0) + 3*K**3*b**2*u(1)**2*x(0) + 2*K**3*b**2*u(1)*u(2)**2*x(0) + K**3*b**2*u(1)*x(0) + K**3*b**2*u(2)**2*x(0) + K**3*b*u(0)*u(1)*x(0) + K**3*b*u(0)*u(2)**2*x(0) + K**3*b*u(1)**3*x(0) + K**3*b*u(1)**2*u(2)**2*x(0) - K**2*b**6*x(0)**2 - 4*K**2*b**5*u(1)*x(0)**2 - 2*K**2*b**5*x(0)**2 - 2*K**2*b**4*u(0)*x(0)**2 - 6*K**2*b**4*u(1)**2*x(0)**2 - 4*K**2*b**4*u(1)*x(0)**2 - 2*K**2*b**4*x(0)**2 - 4*K**2*b**3*u(0)*u(1)*x(0)**2 - 2*K**2*b**3*u(0)*x(0)**2 - 4*K**2*b**3*u(1)**3*x(0)**2 - 2*K**2*b**3*u(1)**2*x(0)**2 - 2*K**2*b**3*u(2)*x(0)**2 - K**2*b**3*x(0)**2 - K**2*b**2*u(0)**2*x(0)**2 - 2*K**2*b**2*u(0)*u(1)**2*x(0)**2 - K**2*b**2*u(1)**4*x(0)

In [116]:
de = x_n.subs(uf.subs(t,desired_iteration), v).diff(v).simplify()

In [117]:
de.atoms()

{-1, 0, 1, 2, K, b, v}

In [118]:
de

2*(b + v)*(K*(b + u(1))**2 + K*u(0) + b*(K - x(0)))*x(0)/K

In [119]:
solve(de, v)

[-b]

In [ ]:
x_n.diff(uf.subs(t,desired_iteration))

In [17]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt

def model(Y,t,K, b, u):

    dRpdt = (b/ K)* Y * (K-Y) + u * Y
    return dRpdt

In [18]:
S = 15
b = 0.8
Y = 0.1
u = 0.2


In [19]:
import ipywidgets

def plot(S, b, u):
    t = np.linspace(0,200,200)
    result = odeint(model,Y,t,args=(S,b, u ))
    fig,ax = plt.subplots()
    ax.plot(t,result[:,0],label='R0=0')

    ax.legend()
    ax.set_xlabel('t')
    ax.set_ylabel('GDP')

ipywidgets.interact(plot, S=(1,100,1), b = (0,1,0.01),  u=(0,1,0.01))

interactive(children=(IntSlider(value=50, description='S', min=1), FloatSlider(value=0.0, description='b', max…

<function __main__.plot(S, b, u)>

In [20]:
import numpy as np
from scipy.optimize import least_squares

# Define the objective function to minimize (residuals)
def objective_function(coefficients, gdp_data, emission_data, gdp_next_data):
    a, b, c, d, e, f, g, h = coefficients
    residuals = gdp_next_data - (a * gdp_data + b * emission_data + c * gdp_data**2 + d * emission_data**2)
    return residuals

# Generate sample data for calibration
gdp_data = np.array([10, 20, 30, 40, 50])  # GDP data at time t
emission_data = np.array([2, 4, 6, 8, 10])  # CO2 emission data at time t
gdp_next_data = np.array([22, 44, 66, 88, 110])  # GDP data at time t+1

# Set initial guess for the coefficients
initial_guess = np.array([1, 1, 1, 1, 1, 1, 1, 1])

# Perform least squares estimation to calibrate the coefficients
result = least_squares(objective_function, initial_guess, args=(gdp_data, emission_data, gdp_next_data))

# Get the calibrated coefficients
calibrated_coefficients = result.x

print("Calibrated Coefficients:")
print("a =", calibrated_coefficients[0])
print("b =", calibrated_coefficients[1])
print("c =", calibrated_coefficients[2])
print("d =", calibrated_coefficients[3])
print("e =", calibrated_coefficients[4])
print("f =", calibrated_coefficients[5])
print("g =", calibrated_coefficients[6])
print("h =", calibrated_coefficients[7])


Calibrated Coefficients:
a = 2.237459701183644
b = -0.18729850591821606
c = -0.2500619461748658
d = 6.2515486543716445
e = 1.0
f = 1.0
g = 1.0
h = 1.0
